# Dataset Builder

Takes raw data and makes a real dataset ready for preprocessing.

## Importing

In [1]:
import pandas as pd

In [3]:
# Get Dataset into pandas dataframe
df_pesagens = pd.read_csv('../data/raw/pesagens2014.csv', sep=',', index_col='PES_ID')
df_rotas = pd.read_csv('../data/raw/rotas.csv', sep=';', index_col='ROTA_ID')

## Info about the raw data

In [4]:
df_pesagens.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123750 entries, 2490322 to 896364
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   _id                 123750 non-null  int64 
 1   LOCDESCARREGO_ID    123750 non-null  int64 
 2   LOCDESCARREGO_DESC  123750 non-null  object
 3   EMP_ID              123750 non-null  int64 
 4   EMP_NOME            123750 non-null  object
 5   ROTA_ID             123749 non-null  object
 6   TPVEICULO_DESC      123750 non-null  object
 7   PES_DATAINI         123750 non-null  object
 8   PES_HRINI           123750 non-null  object
 9   PES_PESOINI         123750 non-null  int64 
 10  PES_DATAFIM         123750 non-null  object
 11  PES_HRFIM           123750 non-null  object
 12  PES_PESOFIM         123750 non-null  int64 
 13  PES_PESOUTIL        123750 non-null  int64 
 14  COLETA_ID           123750 non-null  int64 
 15  COLETA_DESC         123750 non-null  object
 16  E

In [5]:
df_rotas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1228 entries, 01001 to V62
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PERCUSSO_I       1227 non-null   float64
 1   EMP_ID           1228 non-null   int64  
 2   LOCAL_ID         1228 non-null   int64  
 3   ROTA_DESC        1228 non-null   object 
 4   TPCIRCUITO_DESC  1228 non-null   object 
 5   LOCAL_NOME       1228 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 67.2+ KB


In [6]:
df_pesagens.head()

,_id,LOCDESCARREGO_ID,LOCDESCARREGO_DESC,EMP_ID,EMP_NOME,ROTA_ID,TPVEICULO_DESC,PES_DATAINI,PES_HRINI,PES_PESOINI,PES_DATAFIM,PES_HRFIM,PES_PESOFIM,PES_PESOUTIL,COLETA_ID,COLETA_DESC,ESPECCOLETA_ID,ESPECCOLETA_DESC
PES_ID,,,,,,,,,,,,,,,,,,
2490322,1,7,CTR-CANDEIAS,708,VITAL - LOTE 2 CONTRATO 6016/10,0411,CAMINHAO COMPACTADOR,2014-12-31T00:00:00,2016-12-01T23:57:01,21280,2015-01-01T00:00:00,2016-12-01T00:12:06,11640,9640,1,COLETA DOMICILIAR,1,DOMICILIAR
2490321,2,7,CTR-CANDEIAS,708,VITAL - LOTE 2 CONTRATO 6016/10,0410,CAMINHAO COMPACTADOR,2014-12-31T00:00:00,2016-12-01T23:51:26,15040,2015-01-01T00:00:00,2016-12-01T00:02:06,11600,3440,1,COLETA DOMICILIAR,1,DOMICILIAR
2490320,3,7,CTR-CANDEIAS,708,VITAL - LOTE 2 CONTRATO 6016/10,0507,CAMINHAO COMPACTADOR,2014-12-31T00:00:00,2016-12-01T23:41:16,18520,2014-12-31T00:00:00,2016-12-01T23:51:25,11800,6720,1,COLETA DOMICILIAR,1,DOMICILIAR
2490319,4,7,CTR-CANDEIAS,708,VITAL - LOTE 2 CONTRATO 6016/10,0424,CAMINHAO COMPACTADOR,2014-12-31T00:00:00,2016-12-01T23:34:38,19780,2014-12-31T00:00:00,2016-12-01T23:43:43,13520,6260,1,COLETA DOMICILIAR,1,DOMICILIAR
2490317,5,7,CTR-CANDEIAS,702,VITAL ENGENHARIA AMBIENTAL S/A - RASPAGEM,R0113,CAMINHAO CACAMBA,2014-12-31T00:00:00,2016-12-01T23:31:20,15020,2014-12-31T00:00:00,2016-12-01T23:41:16,9790,5230,25,RASPAGEM,50,RASPAGEM


In [7]:
df_rotas.head()

,PERCUSSO_I,EMP_ID,LOCAL_ID,ROTA_DESC,TPCIRCUITO_DESC,LOCAL_NOME
ROTA_ID,,,,,,
01001,1873.0,27,671,COLETA DOM COMPACTADORES,-REA,ZUMBI
01001,1866.0,27,680,COLETA DOM COMPACTADORES,-REA,CORDEIRO
01001,1867.0,27,698,COLETA DOM COMPACTADORES,-REA,IPUTINGA
01002,1697.0,27,744,COLETA DOM COMPACTADORES,-REA,CAXANGA
01003,1835.0,27,728,COLETA DOM COMPACTADORES,-REA,CIDADE UNIVERSITARIA


By looking at both data, as well as looking at the [documentation of the data](http://dados.recife.pe.gov.br/dataset/pesagem-de-coletas-de-residuos), we know that the point of connection for both data is `ROTA_ID` attribute. 

We've gotta be sure that all instances from `pesagens` dataset has a valid `ROTA_ID` that is present on `roteirizacao` dataset, for this, we'll aggregate all valid data and non-valid data will be discarded.

## Merging Rotas Data into Pesagens 

Now we're gonna merge both datasets into just one dataset. This is achieved by using the `merge` method from **pandas**, but, as seen in our tests, if we have duplicate indexes we may see a duplication of our data. 

To fix this, we're gonna drop duplicate indexes present on our data, to make sure the merge is successfull and  no duplicate data is present in the merge.

In [8]:
df_pesagens = df_pesagens.reset_index().drop_duplicates(subset='PES_ID', keep='first').set_index('PES_ID')
df_rotas = df_rotas.reset_index().drop_duplicates(subset='ROTA_ID', keep='first').set_index('ROTA_ID')

Duplicates removed, we can now merge into one dataset.

In [9]:
df = df_pesagens.merge(df_rotas, on='ROTA_ID')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119801 entries, 0 to 119800
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   _id                 119801 non-null  int64  
 1   LOCDESCARREGO_ID    119801 non-null  int64  
 2   LOCDESCARREGO_DESC  119801 non-null  object 
 3   EMP_ID_x            119801 non-null  int64  
 4   EMP_NOME            119801 non-null  object 
 5   ROTA_ID             119801 non-null  object 
 6   TPVEICULO_DESC      119801 non-null  object 
 7   PES_DATAINI         119801 non-null  object 
 8   PES_HRINI           119801 non-null  object 
 9   PES_PESOINI         119801 non-null  int64  
 10  PES_DATAFIM         119801 non-null  object 
 11  PES_HRFIM           119801 non-null  object 
 12  PES_PESOFIM         119801 non-null  int64  
 13  PES_PESOUTIL        119801 non-null  int64  
 14  COLETA_ID           119801 non-null  int64  
 15  COLETA_DESC         119801 non-nul

## Exporting our dataset

Simply we're gonna export into a `.csv`.

In [11]:
df.to_csv('../data/dataset.csv', sep=',', index=True, index_label='PES_ID')